# Self-Driving car

In [ ]:
# Credits: https://github.com/SullyChen/Autopilot-TensorFlow
# Research paper: End to End Learning for Self-Driving Cars by Nvidia. [https://arxiv.org/pdf/1604.07316.pdf]

# NVidia dataset: 72 hrs of video => 72*60*60*30 = 7,776,000 images
# Nvidia blog: https://devblogs.nvidia.com/deep-learning-self-driving-cars/


# Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
# Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB


# If you want to try on a slightly large dataset: 70 minutes of data ~ 223GB
# Refer: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5
# Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed



# Additional Installations:
# pip3 install h5py


# AWS: https://aws.amazon.com/blogs/machine-learning/get-started-with-deep-learning-using-the-aws-deep-learning-ami/

# Youtube:https://www.youtube.com/watch?v=qhUvQiKec2U
# Further reading and extensions: https://medium.com/udacity/teaching-a-machine-to-steer-a-car-d73217f2492c
# More data: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5

In [3]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
sess = tf.InteractiveSession()

L2NormConst = 0.5

train_vars = tf.trainable_variables()

In [5]:
loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [6]:
# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)


In [8]:
# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 15
batch_size = 100



In [ ]:
'''
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply((tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the output

'''

### training DL model

In [9]:
# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Epoch: 0, Step: 0, Loss: 3038.02
Epoch: 0, Step: 10, Loss: 2540.7
Epoch: 0, Step: 20, Loss: 2105.78
Epoch: 0, Step: 30, Loss: 1736.5
Epoch: 0, Step: 40, Loss: 1428.37
Epoch: 0, Step: 50, Loss: 1172.64
Epoch: 0, Step: 60, Loss: 961.882
Epoch: 0, Step: 70, Loss: 788.174
Epoch: 0, Step: 80, Loss: 644.756
Epoch: 0, Step: 90, Loss: 526.55
Epoch: 0, Step: 100, Loss: 429.493
Epoch: 0, Step: 110, Loss: 349.691
Epoch: 0, Step: 120, Loss: 284.253
Epoch: 0, Step: 130, Loss: 230.938
Epoch: 0, Step: 140, Loss: 187.573
Epoch: 0, Step: 150, Loss: 151.14
Epoch: 0, Step: 160, Loss: 122.485
Epoch: 0, Step: 170, Loss: 98.1625
Epoch: 0, Step: 180, Loss: 79.2826
Epoch: 0, Step: 190, Loss: 63.4032
Epoch: 0, Step: 200, Loss: 50.4534
Epoch: 0, Step: 210, Loss: 40.2868
Epoch: 0, Step: 220, Loss: 32.1656
Epoch: 0, Step: 230, Loss: 25.7087
Epoch: 0, Step: 240, Loss: 21.4134
Epoch: 0, Step: 250, Loss: 16.3622
Epoch: 0, Step: 260, Loss: 14.8895
Epoch: 0, Step: 270, Loss: 10.7511
Epoch: 0, Step: 280, Loss: 9.23455


Epoch: 2, Step: 300, Loss: 0.0493443
Epoch: 2, Step: 310, Loss: 0.0417833
Epoch: 2, Step: 320, Loss: 4.02947
Epoch: 2, Step: 330, Loss: 1.57323
Epoch: 2, Step: 340, Loss: 0.0123585
Epoch: 2, Step: 350, Loss: 0.0180971
Epoch: 2, Step: 360, Loss: 0.0641954
Epoch: 2, Step: 370, Loss: 0.0827754
Epoch: 2, Step: 380, Loss: 0.0269411
Epoch: 2, Step: 390, Loss: 0.0896237
Epoch: 2, Step: 400, Loss: 0.10211
Epoch: 2, Step: 410, Loss: 0.00962046
Epoch: 2, Step: 420, Loss: 0.0473798
Epoch: 2, Step: 430, Loss: 0.023555
Epoch: 2, Step: 440, Loss: 0.0211443
Epoch: 2, Step: 450, Loss: 0.0107827
Epoch: 2, Step: 460, Loss: 0.143152
Epoch: 2, Step: 470, Loss: 0.0623368
Epoch: 2, Step: 480, Loss: 0.123718
Epoch: 2, Step: 490, Loss: 0.0977726
Epoch: 2, Step: 500, Loss: 0.035534
Epoch: 2, Step: 510, Loss: 0.0288602
Epoch: 2, Step: 520, Loss: 0.0302137
Epoch: 2, Step: 530, Loss: 0.0269427
Epoch: 2, Step: 540, Loss: 0.0272522
Epoch: 2, Step: 550, Loss: 0.0272533
Epoch: 2, Step: 560, Loss: 0.0274204
Epoch: 2, 

Epoch: 4, Step: 580, Loss: 0.211527
Epoch: 4, Step: 590, Loss: 0.0141498
Epoch: 4, Step: 600, Loss: 0.00960298
Epoch: 4, Step: 610, Loss: 0.0107269
Epoch: 4, Step: 620, Loss: 0.0110177
Epoch: 4, Step: 630, Loss: 0.0103235
Epoch: 4, Step: 640, Loss: 0.525956
Epoch: 4, Step: 650, Loss: 4.91271
Epoch: 4, Step: 660, Loss: 0.0433241
Epoch: 4, Step: 670, Loss: 0.0113092
Epoch: 4, Step: 680, Loss: 0.00784955
Epoch: 4, Step: 690, Loss: 0.0138049
Epoch: 4, Step: 700, Loss: 0.00983139
Epoch: 4, Step: 710, Loss: 0.00784591
Epoch: 4, Step: 720, Loss: 0.00329447
Epoch: 4, Step: 730, Loss: 0.0030745
Epoch: 4, Step: 740, Loss: 0.00300403
Epoch: 4, Step: 750, Loss: 0.00206114
Epoch: 4, Step: 760, Loss: 0.00450816
Epoch: 4, Step: 770, Loss: 0.0406911
Epoch: 4, Step: 780, Loss: 0.0472774
Epoch: 4, Step: 790, Loss: 0.0179809
Epoch: 4, Step: 800, Loss: 0.00129017
Epoch: 4, Step: 810, Loss: 0.00226872
Epoch: 4, Step: 820, Loss: 0.00215415
Epoch: 4, Step: 830, Loss: 0.00174112
Epoch: 4, Step: 840, Loss: 0.0

Epoch: 6, Step: 850, Loss: 0.420788
Epoch: 6, Step: 860, Loss: 0.00201882
Epoch: 6, Step: 870, Loss: 0.0395117
Epoch: 6, Step: 880, Loss: 0.941841
Epoch: 6, Step: 890, Loss: 1.19531
Epoch: 6, Step: 900, Loss: 0.0610248
Epoch: 6, Step: 910, Loss: 0.0802415
Epoch: 6, Step: 920, Loss: 0.0143896
Epoch: 6, Step: 930, Loss: 0.323237
Epoch: 6, Step: 940, Loss: 0.213042
Epoch: 6, Step: 950, Loss: 0.781413
Epoch: 6, Step: 960, Loss: 1.03698
Epoch: 6, Step: 970, Loss: 0.253343
Epoch: 6, Step: 980, Loss: 0.00473398
Epoch: 6, Step: 990, Loss: 0.013963
Epoch: 6, Step: 1000, Loss: 0.0231455
Epoch: 6, Step: 1010, Loss: 0.0297973
Epoch: 6, Step: 1020, Loss: 0.408884
Epoch: 6, Step: 1030, Loss: 0.646578
Epoch: 6, Step: 1040, Loss: 0.246076
Epoch: 6, Step: 1050, Loss: 0.0679289
Model saved in file: ./save\model.ckpt
Epoch: 7, Step: 700, Loss: 0.0478234
Epoch: 7, Step: 710, Loss: 0.0125674
Epoch: 7, Step: 720, Loss: 0.126415
Epoch: 7, Step: 730, Loss: 1.04984
Epoch: 7, Step: 740, Loss: 1.34545
Epoch: 7, 

Epoch: 8, Step: 1110, Loss: 0.0157532
Epoch: 8, Step: 1120, Loss: 0.0152392
Epoch: 8, Step: 1130, Loss: 0.0158414
Epoch: 8, Step: 1140, Loss: 0.0304534
Epoch: 8, Step: 1150, Loss: 0.0243312
Epoch: 8, Step: 1160, Loss: 0.0323701
Epoch: 8, Step: 1170, Loss: 0.00361976
Epoch: 8, Step: 1180, Loss: 0.0337839
Epoch: 8, Step: 1190, Loss: 0.0979641
Epoch: 8, Step: 1200, Loss: 0.186717
Epoch: 8, Step: 1210, Loss: 0.124635
Epoch: 8, Step: 1220, Loss: 0.0334372
Epoch: 8, Step: 1230, Loss: 0.119361
Epoch: 8, Step: 1240, Loss: 0.211833
Epoch: 8, Step: 1250, Loss: 0.321664
Model saved in file: ./save\model.ckpt
Epoch: 9, Step: 900, Loss: 0.0126685
Epoch: 9, Step: 910, Loss: 0.208447
Epoch: 9, Step: 920, Loss: 0.333764
Epoch: 9, Step: 930, Loss: 0.0296486
Epoch: 9, Step: 940, Loss: 0.0453534
Epoch: 9, Step: 950, Loss: 0.00324245
Epoch: 9, Step: 960, Loss: 0.0431088
Epoch: 9, Step: 970, Loss: 0.0220666
Epoch: 9, Step: 980, Loss: 0.729728
Epoch: 9, Step: 990, Loss: 0.287771
Epoch: 9, Step: 1000, Loss: 

Epoch: 10, Step: 1410, Loss: 0.00191804
Epoch: 10, Step: 1420, Loss: 0.000381772
Epoch: 10, Step: 1430, Loss: 0.000636881
Epoch: 10, Step: 1440, Loss: 0.00151248
Epoch: 10, Step: 1450, Loss: 0.00134782
Model saved in file: ./save\model.ckpt
Epoch: 11, Step: 1100, Loss: 0.00105896
Epoch: 11, Step: 1110, Loss: 0.000636083
Epoch: 11, Step: 1120, Loss: 0.000652703
Epoch: 11, Step: 1130, Loss: 0.000668846
Epoch: 11, Step: 1140, Loss: 0.00189881
Epoch: 11, Step: 1150, Loss: 0.0542245
Epoch: 11, Step: 1160, Loss: 0.192101
Epoch: 11, Step: 1170, Loss: 5.18973
Epoch: 11, Step: 1180, Loss: 0.0125918
Epoch: 11, Step: 1190, Loss: 0.0012807
Epoch: 11, Step: 1200, Loss: 0.0382529
Epoch: 11, Step: 1210, Loss: 0.0717636
Epoch: 11, Step: 1220, Loss: 0.0691108
Epoch: 11, Step: 1230, Loss: 0.00505631
Epoch: 11, Step: 1240, Loss: 0.129506
Epoch: 11, Step: 1250, Loss: 0.0500563
Epoch: 11, Step: 1260, Loss: 0.001602
Epoch: 11, Step: 1270, Loss: 0.0574705
Epoch: 11, Step: 1280, Loss: 0.00981877
Epoch: 11, St

Epoch: 13, Step: 1310, Loss: 0.476401
Epoch: 13, Step: 1320, Loss: 0.117622
Epoch: 13, Step: 1330, Loss: 0.0114783
Epoch: 13, Step: 1340, Loss: 0.00617206
Epoch: 13, Step: 1350, Loss: 0.00168822
Epoch: 13, Step: 1360, Loss: 0.0105284
Epoch: 13, Step: 1370, Loss: 0.0330249
Epoch: 13, Step: 1380, Loss: 0.0468767
Epoch: 13, Step: 1390, Loss: 0.00292681
Epoch: 13, Step: 1400, Loss: 0.0984827
Epoch: 13, Step: 1410, Loss: 0.264059
Epoch: 13, Step: 1420, Loss: 0.242569
Epoch: 13, Step: 1430, Loss: 0.216372
Epoch: 13, Step: 1440, Loss: 0.0158042
Epoch: 13, Step: 1450, Loss: 0.01376
Epoch: 13, Step: 1460, Loss: 0.0111437
Epoch: 13, Step: 1470, Loss: 0.0106696
Epoch: 13, Step: 1480, Loss: 0.0177963
Epoch: 13, Step: 1490, Loss: 2.93707
Epoch: 13, Step: 1500, Loss: 2.5945
Epoch: 13, Step: 1510, Loss: 0.0116608
Epoch: 13, Step: 1520, Loss: 0.00411481
Epoch: 13, Step: 1530, Loss: 0.0036674
Epoch: 13, Step: 1540, Loss: 0.0102198
Epoch: 13, Step: 1550, Loss: 0.0034668
Epoch: 13, Step: 1560, Loss: 0.00

### testing the model

In [11]:


import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)

i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


INFO:tensorflow:Restoring parameters from save/model.ckpt
